In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Union and JOins").master("local[*]").getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/04/12 04:44:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/12 04:44:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/12 04:44:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/04/12 04:44:52 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/04/12 04:44:52 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [5]:
emp_data_1 = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"]
]

emp_data_2 = [
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

In [8]:
#creating a schema
employee_schema="employee_id string,department_id string,name string,age string,gender string,salary string,hire_date string"

In [13]:
df1=spark.createDataFrame(emp_data_1,employee_schema)
df2=spark.createDataFrame(data=emp_data_2,schema=employee_schema)
df1.printSchema()

df2.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [14]:
schema_in_structtype=df1.schema
schema_in_structtype

StructType([StructField('employee_id', StringType(), True), StructField('department_id', StringType(), True), StructField('name', StringType(), True), StructField('age', StringType(), True), StructField('gender', StringType(), True), StructField('salary', StringType(), True), StructField('hire_date', StringType(), True)])

In [15]:
df1.union(df2).show() #show is a action

[Stage 5:===============================>                          (6 + 5) / 11]

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [16]:
df1.columns

['employee_id',
 'department_id',
 'name',
 'age',
 'gender',
 'salary',
 'hire_date']

In [17]:
#select employee_id,name,age,salary from emp
from pyspark.sql.functions import col,expr
emp_data=df1.select("employee_id",col("department_id"),expr("name"),df1.age,df1.salary)

In [18]:
emp_data.show()

+-----------+-------------+-------------+---+------+
|employee_id|department_id|         name|age|salary|
+-----------+-------------+-------------+---+------+
|        001|          101|     John Doe| 30| 50000|
|        002|          101|   Jane Smith| 25| 45000|
|        003|          102|    Bob Brown| 35| 55000|
|        004|          102|    Alice Lee| 28| 48000|
|        005|          103|    Jack Chan| 40| 60000|
|        006|          103|    Jill Wong| 32| 52000|
|        007|          101|James Johnson| 42| 70000|
|        008|          102|     Kate Kim| 29| 51000|
|        009|          103|      Tom Tan| 33| 58000|
|        010|          104|     Lisa Lee| 27| 47000|
+-----------+-------------+-------------+---+------+



In [22]:
#casting-> converting from one datatype to another datatype

emp_data.printSchema()
emp_data_casted=emp_data.select(expr("employee_id as emp_id"),df1.name,expr("cast(age as int) as age"),expr("cast(salary as long) as salary"))
emp_data_casted.printSchema()
emp_data_casted.show()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: string (nullable = true)

root
 |-- emp_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: long (nullable = true)

+------+-------------+---+------+
|emp_id|         name|age|salary|
+------+-------------+---+------+
|   001|     John Doe| 30| 50000|
|   002|   Jane Smith| 25| 45000|
|   003|    Bob Brown| 35| 55000|
|   004|    Alice Lee| 28| 48000|
|   005|    Jack Chan| 40| 60000|
|   006|    Jill Wong| 32| 52000|
|   007|James Johnson| 42| 70000|
|   008|     Kate Kim| 29| 51000|
|   009|      Tom Tan| 33| 58000|
|   010|     Lisa Lee| 27| 47000|
+------+-------------+---+------+



In [24]:
#filter the employee based on the age above 30

emp_above_30=emp_data_casted.filter("age>30")
emp_above_30=emp_data_casted.where("age>30")
emp_above_30.show() #show is a action

+------+-------------+---+------+
|emp_id|         name|age|salary|
+------+-------------+---+------+
|   003|    Bob Brown| 35| 55000|
|   005|    Jack Chan| 40| 60000|
|   006|    Jill Wong| 32| 52000|
|   007|James Johnson| 42| 70000|
|   009|      Tom Tan| 33| 58000|
+------+-------------+---+------+



In [28]:
emp_above_30.rdd.getNumPartitions()

8

In [33]:
#to write the above result into a file like csv(Action)
emp_above_30.coalesce(1).write.format("csv").save("emp_above302.csv")

In [29]:
schema_test="name string,age int"
#I want to convert this into StructType
# StrutType[StructField("name",StringType(),StructField("age",IntegerType()))]
from pyspark.sql.types import _parse_datatype_string
schema_sparktype=_parse_datatype_string(schema_test)
schema_sparktype

StructType([StructField('name', StringType(), True), StructField('age', IntegerType(), True)])

In [34]:
disordered_df=df1.select(df1.age,df1.salary,df1.employee_id,df1.hire_date,df1.department_id,df1.gender,df1.name)
disordered_df.printSchema()

root
 |-- age: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- employee_id: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)



In [35]:
df1.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [ ]:
df1.union(disordered_df).show()#give wrong union data if the datafram column is not in sequence

In [38]:
#if your dataframe is not in sequence of another dataframe, then use unionByName
df1.unionByName(disordered_df).show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|     

In [52]:
dept_data=[
    ["101","Sales","NYC","US"],
    ["102","Marekting","NewJersy","US"],
    ["103","Product Development","Delhi","India"],
    ["104","HR","Chennai","India"],
    ["105","Finance","Hyderabad","India"]
]
dept_schema="department_id string,dept_name string,city string,country string"


In [53]:
dept_df=spark.createDataFrame(dept_data,dept_schema)
dept_df.show()

+-------------+-------------------+---------+-------+
|department_id|          dept_name|     city|country|
+-------------+-------------------+---------+-------+
|          101|              Sales|      NYC|     US|
|          102|          Marekting| NewJersy|     US|
|          103|Product Development|    Delhi|  India|
|          104|                 HR|  Chennai|  India|
|          105|            Finance|Hyderabad|  India|
+-------------+-------------------+---------+-------+



In [54]:
df1.rdd.getNumPartitions()

8

In [55]:
dept_df.rdd.getNumPartitions()

8

In [59]:
df1_reduced_partitions=df1.coalesce(4)
df1_reduced_partitions.rdd.getNumPartitions()
dept_df_reduced_partitions=dept_df.coalesce(4)
dept_df_reduced_partitions.rdd.getNumPartitions()

4

In [62]:
#select e.emp_name,d.department_name,department_id,salary 
#from emp e inner join dept d on emp.department_id=d.department_id

df_joined=df1_reduced_partitions.alias("e").join(dept_df_reduced_partitions.alias("d"),how="inner",on=df1_reduced_partitions.department_id==dept_df_reduced_partitions.department_id)


In [66]:
df_joined.select("e.name","d.dept_name","e.salary","d.department_id").show()

[Stage 69:=============>    (3 + 1) / 4][Stage 70:=========>        (2 + 2) / 4]

+-------------+-------------------+------+-------------+
|         name|          dept_name|salary|department_id|
+-------------+-------------------+------+-------------+
|     John Doe|              Sales| 50000|          101|
|   Jane Smith|              Sales| 45000|          101|
|James Johnson|              Sales| 70000|          101|
|    Bob Brown|          Marekting| 55000|          102|
|    Alice Lee|          Marekting| 48000|          102|
|     Kate Kim|          Marekting| 51000|          102|
|    Jack Chan|Product Development| 60000|          103|
|    Jill Wong|Product Development| 52000|          103|
|      Tom Tan|Product Development| 58000|          103|
|     Lisa Lee|                 HR| 47000|          104|
+-------------+-------------------+------+-------------+



In [68]:
#select e.emp_name,d.department_name,department_id,salary 
#from emp e left outer join dept d on emp.department_id=d.department_id

df_left_joined=df1_reduced_partitions.alias("e").join(dept_df_reduced_partitions.alias("d"),how="left_outer",on=df1_reduced_partitions.department_id==dept_df_reduced_partitions.department_id)
df_left_joined.select("e.name","d.dept_name","e.salary","d.department_id").show()

+-------------+-------------------+------+-------------+
|         name|          dept_name|salary|department_id|
+-------------+-------------------+------+-------------+
|     John Doe|              Sales| 50000|          101|
|   Jane Smith|              Sales| 45000|          101|
|    Bob Brown|          Marekting| 55000|          102|
|    Alice Lee|          Marekting| 48000|          102|
|    Jack Chan|Product Development| 60000|          103|
|James Johnson|              Sales| 70000|          101|
|    Jill Wong|Product Development| 52000|          103|
|     Lisa Lee|                 HR| 47000|          104|
|     Kate Kim|          Marekting| 51000|          102|
|      Tom Tan|Product Development| 58000|          103|
+-------------+-------------------+------+-------------+



In [70]:
#select e.emp_name,d.department_name,department_id,salary 
#from emp e right outer join dept d on emp.department_id=d.department_id

df_right_joined=df1_reduced_partitions.alias("e").join(dept_df_reduced_partitions.alias("d"),how="full_outer",on=df1_reduced_partitions.department_id==dept_df_reduced_partitions.department_id)
df_right_joined.select("e.name","d.dept_name","e.salary","d.department_id").show()

[Stage 86:=========>        (2 + 2) / 4][Stage 87:>                 (0 + 4) / 4]

+-------------+-------------------+------+-------------+
|         name|          dept_name|salary|department_id|
+-------------+-------------------+------+-------------+
|     John Doe|              Sales| 50000|          101|
|   Jane Smith|              Sales| 45000|          101|
|James Johnson|              Sales| 70000|          101|
|    Bob Brown|          Marekting| 55000|          102|
|    Alice Lee|          Marekting| 48000|          102|
|     Kate Kim|          Marekting| 51000|          102|
|    Jack Chan|Product Development| 60000|          103|
|    Jill Wong|Product Development| 52000|          103|
|      Tom Tan|Product Development| 58000|          103|
|     Lisa Lee|                 HR| 47000|          104|
|         null|            Finance|  null|          105|
+-------------+-------------------+------+-------------+

